In [18]:
%cd /content

!rm -rf /content/conda
!rm -rf /content/nmt
!rm Miniconda3-py37_4.12.0-Linux-x86_64.sh

!wget -q https://repo.anaconda.com/miniconda/Miniconda3-py37_4.12.0-Linux-x86_64.sh
!bash Miniconda3-py37_4.12.0-Linux-x86_64.sh -b -p /content/conda

!/content/conda/bin/conda install -y tensorflow-gpu=1.15 cudatoolkit=10.0 cudnn=7.6.5

!git clone https://github.com/tensorflow/nmt
%cd /content/nmt
!sh scripts/download_iwslt15.sh /tmp/nmt_data

!mkdir -p /content/nmt_model

print("=== CÀI ĐẶT HOÀN TẤT ===")

/content
rm: cannot remove 'Miniconda3-py37_4.12.0-Linux-x86_64.sh': No such file or directory
PREFIX=/content/conda
Unpacking payload ...
Solving environment: / - \ done

## Package Plan ##

  environment location: /content/conda

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - _openmp_mutex==4.5=1_gnu
    - brotlipy==0.7.0=py37h27cfd23_1003
    - ca-certificates==2022.3.29=h06a4308_1
    - certifi==2021.10.8=py37h06a4308_2
    - cffi==1.15.0=py37hd667e15_1
    - charset-normalizer==2.0.4=pyhd3eb1b0_0
    - colorama==0.4.4=pyhd3eb1b0_0
    - conda-content-trust==0.1.1=pyhd3eb1b0_0
    - conda-package-handling==1.8.1=py37h7f8727e_0
    - conda==4.12.0=py37h06a4308_0
    - cryptography==36.0.0=py37h9ce1e76_0
    - idna==3.3=pyhd3eb1b0_0
    - ld_impl_linux-64==2.35.1=h7274673_9
    - libffi==3.3=he6710b0_2
    - libgcc-ng==9.3.0=h5101ec6_17
    - libgomp==9.3.0=h5101ec6_17
    - libstdcxx-ng==9.3.0=hd4cf53a_17
    - ncurses==6.3=h7f8727e_2
    - openssl==1.1.1n=h7f872

In [25]:
!rm -rf /tmp/nmt_data
!mkdir -p /tmp/nmt_data

%cd /tmp/nmt_data

!wget https://github.com/stefan-it/nmt-en-vi/raw/master/data/train-en-vi.tgz
!tar -xzvf train-en-vi.tgz
!mv train.en train.en
!mv train.vi train.vi

!wget https://github.com/stefan-it/nmt-en-vi/raw/master/data/dev-2012-en-vi.tgz
!tar -xzvf dev-2012-en-vi.tgz
!mv tst2012.en tst2012.en
!mv tst2012.vi tst2012.vi

!wget https://github.com/stefan-it/nmt-en-vi/raw/master/data/test-2013-en-vi.tgz
!tar -xzvf test-2013-en-vi.tgz
!mv tst2013.en tst2013.en
!mv tst2013.vi tst2013.vi

!head -n 3 train.en

/tmp/nmt_data
--2025-11-17 06:06:16--  https://github.com/stefan-it/nmt-en-vi/raw/master/data/train-en-vi.tgz
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/stefan-it/nmt-en-vi/master/data/train-en-vi.tgz [following]
--2025-11-17 06:06:16--  https://raw.githubusercontent.com/stefan-it/nmt-en-vi/master/data/train-en-vi.tgz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9903559 (9.4M) [application/octet-stream]
Saving to: ‘train-en-vi.tgz’

train-en-vi.tgz     100%[===================>]   9.44M  --.-KB/s    in 0.03s   

2025-11-17 06:06:17 (361 MB/s) - ‘train-en-vi.tgz’ saved [9903559/9903559]



In [28]:
import os
import collections

def create_vocab(data_file, vocab_file):
    print(f"Đang tạo vocab từ {data_file} ...")

    words = []
    with open(data_file, 'r', encoding='utf-8') as f:
        for line in f:
            words.extend(line.strip().split())

    counter = collections.Counter(words)
    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))

    words, _ = list(zip(*count_pairs))

    special_tokens = ["<unk>", "<s>", "</s>"]
    vocab_words = special_tokens + list(words)

    with open(vocab_file, 'w', encoding='utf-8') as f:
        for w in vocab_words:
            f.write(w + "\n")

    print(f"-> Đã tạo xong: {vocab_file} (Số từ: {len(vocab_words)})")

data_dir = "/tmp/nmt_data"
create_vocab(os.path.join(data_dir, "train.en"), os.path.join(data_dir, "vocab.en"))
create_vocab(os.path.join(data_dir, "train.vi"), os.path.join(data_dir, "vocab.vi"))

Đang tạo vocab từ /tmp/nmt_data/train.en ...
-> Đã tạo xong: /tmp/nmt_data/vocab.en (Số từ: 54172)
Đang tạo vocab từ /tmp/nmt_data/train.vi ...
-> Đã tạo xong: /tmp/nmt_data/vocab.vi (Số từ: 25618)


In [45]:
%cd /content/nmt
!/content/conda/bin/python -m nmt.nmt \
    --src=en --tgt=vi \
    --vocab_prefix=/tmp/nmt_data/vocab  \
    --train_prefix=/tmp/nmt_data/train \
    --dev_prefix=/tmp/nmt_data/tst2012  \
    --test_prefix=/tmp/nmt_data/tst2013 \
    --out_dir=/content/nmt_model_final \
    --num_train_steps=1000 \
    --steps_per_stats=100 \
    --num_layers=2 \
    --num_units=128 \
    --dropout=0.2 \
    --metrics=bleu \

/content/nmt

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W1117 07:41:28.357489 136190458246272 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

# Job id 0

W1117 07:41:28.809377 136190458246272 module_wrapper.py:139] From /content/nmt/nmt/nmt.py:629: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2025-11-17 07:41

In [46]:
!/content/conda/bin/python -m nmt.nmt \
    --out_dir=/content/nmt_model_final \
    --inference_input_file=/content/cau_hoi.txt \
    --inference_output_file=/content/ket_qua_full.txt \
    --vocab_prefix=/tmp/nmt_data/vocab


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W1117 08:46:12.185178 137666087871616 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

# Job id 0

W1117 08:46:12.677592 137666087871616 module_wrapper.py:139] From /content/nmt/nmt/nmt.py:629: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2025-11-17 08:46:12.678786: W

In [47]:
!cat /content/ket_qua_full.txt

Tôi không phải là những gì .
Tôi không thể làm việc .
